In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

compute_cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, use it.")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
### YOUR CODE HERE ###

ps = RandomParameterSampling({"--C" : choice(0.1,0.5,1,2,2.5,3),
                              "--max_iter" : choice(20,30,50,100,150,200)
                             })

# Specify a Policy
### YOUR CODE HERE ###

policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1, delay_evaluation= 5 )

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###

est = SKLearn(source_directory = './training', entry_script = 'train.py', compute_target = compute_cluster_name)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###

primary_metric_name = 'Accuracy'

primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

hyperdrive_config = HyperDriveConfig(hyperparameter_sampling= ps,
                                     primary_metric_name= primary_metric_name,
                                     max_total_runs= 100,
                                     max_concurrent_runs= 4,
                                     primary_metric_goal = primary_metric_goal,
                                     policy= policy,
                                     estimator = est)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()

hyperdrive_run.wait_for_completion(show_output=True)


In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("best run details :",best_run.get_details())
print('\n')
print("best run file names :",best_run.get_file_names())
print('\n')
print("best run metrics :",best_run.get_metrics())

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [ ]:
import pandas as pd
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core import Dataset

# Use the clean_data function to clean your data.
### YOUR DATA OBJECT HERE ###
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

training_df = pd.concat([x_train,y_train],axis=1)

training_df.to_csv('training/training_df.csv')
ds1 = ws.get_default_datastore()
ds1.upload(src_dir='training',target_path='training')

train_ds = Dataset.Tabular.from_delimited_files(ds1.path('training/training_df.csv'))

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=train_ds,
    label_column_name='y',
    n_cross_validations=5,
    compute_target = compute_target,
    enable_early_stopping = True)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(config= automl_config, show_output= True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_metrics()

In [ ]:
fitted_model

In [ ]:
compute_target.delete()